In [1]:
import pandas as pd, numpy as np
import zipfile
import glob

! whoami
! date

zmbc
Thu Sep 29 09:32:28 PDT 2022


In [2]:
# 2019 data does not work!
! mkdir -p ../data/raw/acs_2019_5yr
! [ ! -e ../data/raw/acs_2019_5yr/csv_pus.zip ] && wget https://www2.census.gov/programs-surveys/acs/data/pums/2019/5-Year/csv_pus.zip -P ../data/raw/acs_2019_5yr

In [3]:
# Neither does 2018!
! mkdir -p ../data/raw/acs_2018_5yr
! [ ! -e ../data/raw/acs_2018_5yr/csv_pus.zip ] && wget https://www2.census.gov/programs-surveys/acs/data/pums/2018/5-Year/csv_pus.zip -P ../data/raw/acs_2018_5yr

In [4]:
! mkdir -p ../data/raw/acs_2020_5yr
! [ ! -e ../data/raw/acs_2020_5yr/csv_pus.zip ] && wget https://www2.census.gov/programs-surveys/acs/data/pums/2020/5-Year/csv_pus.zip -P ../data/raw/acs_2020_5yr

In [5]:
try:
    with zipfile.ZipFile('../data/raw/acs_2019_5yr/csv_pus.zip', 'r') as zip_ref:
        zip_ref.extractall('../data/raw/acs_2019_5yr')
except zipfile.BadZipFile:
    print('2019 5-year zip file malformed')

2019 5-year zip file malformed


In [6]:
try:
    with zipfile.ZipFile('../data/raw/acs_2018_5yr/csv_pus.zip', 'r') as zip_ref:
        zip_ref.extractall('../data/raw/acs_2018_5yr')
except zipfile.BadZipFile:
    print('2018 5-year zip file malformed')

2018 5-year zip file malformed


In [7]:
with zipfile.ZipFile('../data/raw/acs_2020_5yr/csv_pus.zip', 'r') as zip_ref:
    zip_ref.extractall('../data/raw/acs_2020_5yr')

In [8]:
%%time

acs_2020_5yr = pd.concat([pd.read_csv(fname) for fname in glob.glob('../data/raw/acs_2020_5yr/psam_pus*.csv')])
len(acs_2020_5yr)

/ihme/code/central_comp/miniconda/envs/v156/lib/python3.7/site-packages/ipykernel_launcher.py:1: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  """Entry point for launching an IPython kernel.


CPU times: user 4min 16s, sys: 59.4 s, total: 5min 16s
Wall time: 5min 16s


15441673

In [9]:
acs_2020_5yr.to_hdf('../data/acs_2020_5yr.hdf', key='acs', mode='w')

/ihme/code/central_comp/miniconda/envs/v156/lib/python3.7/site-packages/pandas/core/generic.py:2718: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->block2_values] [items->Index(['RT', 'SERIALNO', 'NAICSP', 'SOCP'], dtype='object')]

  encoding=encoding,


In [10]:
! mkdir -p ../data/raw/acs_2019_1yr
! [ ! -e ../data/raw/acs_2019_1yr/csv_pus.zip ] && wget https://www2.census.gov/programs-surveys/acs/data/pums/2019/1-Year/csv_pus.zip -P ../data/raw/acs_2019_1yr

In [11]:
with zipfile.ZipFile('../data/raw/acs_2019_1yr/csv_pus.zip', 'r') as zip_ref:
    zip_ref.extractall('../data/raw/acs_2019_1yr')

In [12]:
%%time

acs_2019_1yr = pd.concat([pd.read_csv(fname) for fname in glob.glob('../data/raw/acs_2019_1yr/psam_pus*.csv')])
len(acs_2019_1yr)

CPU times: user 57.8 s, sys: 10.7 s, total: 1min 8s
Wall time: 1min 8s


3239553

In [13]:
acs_2019_1yr.to_hdf('../data/acs_2019_1yr.hdf', key='acs', mode='w')

/ihme/code/central_comp/miniconda/envs/v156/lib/python3.7/site-packages/pandas/core/generic.py:2718: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block2_values] [items->Index(['RT', 'SERIALNO', 'NAICSP', 'SOCP'], dtype='object')]

  encoding=encoding,
